In [64]:
"""
This is just to enjoy/play this interesting library (BERTopics) and their grapsh
Trying to use PyLDAvis visualization for topics modeling.

"""

'\nThis is just to enjoy/play this interesting library (BERTopics) and their grapsh\n'

In [65]:
name_notebook = 'bertopics_pyldavis'

In [66]:
#from ipywidgets import  IProgress

In [67]:
%load_ext autoreload
%autoreload 2
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import os
import pyLDAvis

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
# For a sorter and faster version
sort_version = True

In [69]:
data_original = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))


In [70]:

if sort_version:
    nr_topics = 20
    data = {}
    number_to_select = 5000
    for key in data_original.keys():
        print(key)
        data[key] = data_original[key][:number_to_select]
else:
    nr_topics = 'auto'
    data = data_original.copy()
docs = data['data'] 

data
filenames
target_names
target
DESCR


In [ ]:
%%time
topic_model = BERTopic(nr_topics = nr_topics, 
                       calculate_probabilities=True, 
                       verbose = True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

2022-01-01 15:00:54,694 - BERTopic - Transformed documents to Embeddings


# Create function to use visualization as in PyLDAVis
From https://github.com/MaartenGr/BERTopic/issues/196

In [ ]:
# Prepare data for PyLDAVis
top_n = 5
R = 10

topic_term_dists = topic_model.c_tf_idf.toarray()[:top_n+1, ]
new_probs = probs[:, :top_n]
outlier = np.array(1 - new_probs.sum(axis=1)).reshape(-1, 1)
doc_topic_dists = np.hstack((new_probs, outlier))
doc_lengths = [len(doc) for doc in docs]
vocab = [word for word in topic_model.vectorizer_model.vocabulary_.keys()]
term_frequency = [topic_model.vectorizer_model.vocabulary_[word] for word in vocab]

data = {'topic_term_dists': topic_term_dists,
        'doc_topic_dists': doc_topic_dists,
        'doc_lengths': doc_lengths,
        'vocab': vocab,
        'term_frequency': term_frequency}

# Visualize using pyLDAvis
vis_data= pyLDAvis.prepare(**data,R=R, n_jobs = 1, mds='mmds')
pyLDAvis.display(vis_data)

In [ ]:
pyLDAvis.save_html(vis_data,"pyldavis.html")

In [ ]:
# to save
os.system("jupytext --output {}.py {}.ipynb".format(name_notebook,name_notebook))